# メタデータフィルタリングについて体験してみよう

[Retrieve と RetrieveAndGenerate の違い](./02.ipynb) で利用したユーザープロンプトを見返してみます。

In [ ]:
import json
import boto3

KNOWLEDGEBASE_ID = "" # ナレッジベース ID を記載
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_agent = boto3.client("bedrock-agent-runtime")
prompt = "私はエンジニアです。オンボーディング初日に必要なことを教えてください。"

response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": 'KNOWLEDGE_BASE',
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn
        },
    },
)

print(response["output"]["text"])

「**私はエンジニアです。**」と名乗って質問する人がどれほどいるでしょうか。

所属する部門などユーザーに紐づいた（パーソナライズ）されたデータを出し分けたいケースを想定してみます。

## メタデータフィルターの利用

Knowledge bases for Amazon Bedrock では、インデックスに登録するデータに対してメタデータを付与できます。

`各ドキュメントの名前.metadata.json` のような形式で同じディレクトリ階層にメタデータファイルを配置します。

```bash
tree .
.
├── README.md
├── all
│   ├── change-account.md
│   ├── change-account.md.metadata.json
│   ├── expense.md
│   ├── expense.md.metadata.json
│   ├── moved.md
│   └── moved.md.metadata.json
├── engineer
│   ├── 01_setup.md
│   └── 01_setup.md.metadata.json
└── sales
    ├── 01_setup.md
    └── 01_setup.md.metadata.json
```

メタデータの値には、文字列、数字、 Bool を設定できます。

```json
{
	"metadataAttributes": {
		"target": "engineer",
		"year": 2024,
		"for_managers": false
	}
}
```

ベクトルデータベースによっては、カラム追加など事前にメタデータに登録するデータのスキーマを定義する必要があるため注意が必要です。

Open Search Serverless の場合はメタデータが自動で適用されますが、インデックスを IaC で管理しているケースの場合は事前に登録が必要です。

```terraform
########################################################
# Index
########################################################
resource "opensearch_index" "this" {
  name          = "${local.prefix}-vector-index"
  index_knn     = true
  force_destroy = true
  mappings = jsonencode({
    properties = {
      AMAZON_BEDROCK_METADATA = {
        type  = "text",
        index = false
      },
      AMAZON_BEDROCK_TEXT_CHUNK = {
        type = "text",
        # analyzer = "custom_kuromoji_analyzer" # Chapter 2 で利用するカスタムアナライザーの設定
      },
      "${local.prefix}-vector" = {
        type      = "knn_vector",
        dimension = var.knowledge_bases.embeddings_model_dimensions,
        method = {
          engine     = "faiss",
          space_type = "l2",
          name       = "hnsw"
          parameters = {}
        }
      },
      for_managers = {
        type = "boolean"
      },
      id = {
        fields = {
          keyword = {
            ignore_above = 256
            type         = "keyword"
          }
        }
        type = "text"
      },
      target = {
        fields = {
          keyword = {
            ignore_above = 256
            type         = "keyword"
          }
        }
        type = "text"
      },
      x-amz-bedrock-kb-data-source-id = {
        fields = {
          keyword = {
            ignore_above = 256
            type         = "keyword"
          }
        }
        type = "text"
      },
      x-amz-bedrock-kb-source-uri = {
        fields = {
          keyword = {
            ignore_above = 256
            type         = "keyword"
          }
        }
        type = "text"
      }
      year = {
        type = "long"
      }
    }
  })
  depends_on = [
    aws_opensearchserverless_security_policy.this_network,
    aws_opensearchserverless_security_policy.this_encryption,
    aws_opensearchserverless_access_policy.this_data
  ]
}
```

メタデータフィルターの利用は RetrieveAndGenerate API の場合、 `retrievalConfiguration` から選択可能です。各職種ごとに資料の出し分けをしてみましょう。

## 営業向け

In [ ]:
import json
import boto3

KNOWLEDGEBASE_ID = "" # ナレッジベース ID を記載
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_agent = boto3.client("bedrock-agent-runtime")

prompt = "入社初日何をしますか？"
role = "sales"

response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "filter": {
                        "equals": {
													"key": "target",
													"value": role
                        }
										}
                }
            }
        },
    },
)

print(response["output"]["text"])

## エンジニア向け

In [ ]:
prompt = "入社初日何をしますか？"
role = "engineer"

response = bedrock_agent.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn,
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "filter": {
                        "equals": {
													"key": "target",
													"value": role
                        }
										}
                }
            }
        },
    },
)

print(response["output"]["text"])

コンテンツの出し分けはうまくいきましたでしょうか？

実際に組み込むとなれば、ユーザーの属性値を認証・認可フェーズで取得し、ナレッジベースへ渡すようなケースが考えられます。

# まとめ

このセクションでは次のことを学びました。

- ユーザー情報に基づく、コンテンツの出し分けはメタデータフィルタリングを使用する
- メタデータは `ファイル名.metadata.json` をコンテンツと同じディレクトリ階層に配置する
- ベクトルデータベースによっては事前にメタデータのスキーマを定義する必要がある